In [69]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


READ AND PREPROCESS DATA

In [125]:
df =pd.read_csv("output.csv")

In [126]:
df.shape

(45, 2)

In [127]:
df.head()

,Bot_Cell,Crew_Cell
0,"(6, 5)","(0, 7)"
1,"(5, 6)","(0, 8)"
2,"(4, 7)","(0, 7)"
3,"(3, 8)","(1, 7)"
4,"(2, 8)","(1, 6)"


In [128]:
# Extract x, y, p, and q values from the existing columns
df['x'] = df['Bot_Cell'].apply(lambda x: int(x.split(',')[0].strip('()')))
df['y'] = df['Bot_Cell'].apply(lambda x: int(x.split(',')[1].strip('()')))
df['p'] = df['Crew_Cell'].apply(lambda x: int(x.split(',')[0].strip('()')))
df['q'] = df['Crew_Cell'].apply(lambda x: int(x.split(',')[1].strip('()')))

# Calculate the distance
df['Distance_from_bot_to_crew'] = abs(df['x'] - df['p']) + abs(df['y'] - df['q'])
df['Distance_from_bot_to_teleport'] = abs(df['x'] - 5) + abs(df['y'] - 5)
df['Distance_from_crew_to_teleport'] = abs(5 - df['p']) + abs(5 - df['q'])
#Drop the intermediate columns x, y, p, and q if needed
df.drop(['x', 'y', 'p', 'q'], axis=1, inplace=True)



In [129]:
df['Bot_Move'] = df['Bot_Cell'].shift(-1)

In [130]:
df.head()

,Bot_Cell,Crew_Cell,Distance_from_bot_to_crew,Distance_from_bot_to_teleport,Distance_from_crew_to_teleport,Bot_Move
0,"(6, 5)","(0, 7)",8,1,7,"(5, 6)"
1,"(5, 6)","(0, 8)",7,1,8,"(4, 7)"
2,"(4, 7)","(0, 7)",4,3,7,"(3, 8)"
3,"(3, 8)","(1, 7)",3,5,6,"(2, 8)"
4,"(2, 8)","(1, 6)",3,6,5,"(1, 7)"


Encodding the data

In [131]:
one_hot_encoded_df = df.copy()
one_hot_encoded_df = pd.get_dummies(df)
one_hot_encoded_df.head()

,Distance_from_bot_to_crew,Distance_from_bot_to_teleport,Distance_from_crew_to_teleport,"Bot_Cell_(0, 3)","Bot_Cell_(0, 4)","Bot_Cell_(0, 5)","Bot_Cell_(0, 6)","Bot_Cell_(0, 7)","Bot_Cell_(1, 3)","Bot_Cell_(1, 4)",...,"Bot_Move_(5, 2)","Bot_Move_(5, 6)","Bot_Move_(6, 2)","Bot_Move_(7, 1)","Bot_Move_(7, 5)","Bot_Move_(8, 2)","Bot_Move_(8, 5)","Bot_Move_(9, 1)","Bot_Move_(9, 5)","Bot_Move_(9, 6)"
0,8,1,7,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
1,7,1,8,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,4,3,7,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,3,5,6,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,3,6,5,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [132]:
label_encoder = LabelEncoder()
label_encoded_df = df.copy()
# Iterate over each column in the DataFrame
for column in label_encoded_df.columns:
    # Check if the column is categorical (i.e., dtype is object)
    if label_encoded_df[column].dtype == 'object':
        # Fit label encoder and transform values for the current column
        label_encoded_df[column] = label_encoder.fit_transform(label_encoded_df[column])

In [133]:
label_encoded_df.head()

,Bot_Cell,Crew_Cell,Distance_from_bot_to_crew,Distance_from_bot_to_teleport,Distance_from_crew_to_teleport,Bot_Move
0,27,2,8,1,7,25
1,25,3,7,1,8,23
2,23,2,4,3,7,20
3,20,8,3,5,6,17
4,17,7,3,6,5,9


In [145]:

correlation_matrix2 = label_encoded_df.corr()
correlation_matrix1 = one_hot_encoded_df.corr()

In [146]:
correlation_matrix1

,Distance_from_bot_to_crew,Distance_from_bot_to_teleport,Distance_from_crew_to_teleport,"Bot_Cell_(0, 3)","Bot_Cell_(0, 4)","Bot_Cell_(0, 5)","Bot_Cell_(0, 6)","Bot_Cell_(0, 7)","Bot_Cell_(1, 3)","Bot_Cell_(1, 4)",...,"Bot_Move_(5, 2)","Bot_Move_(5, 6)","Bot_Move_(6, 2)","Bot_Move_(7, 1)","Bot_Move_(7, 5)","Bot_Move_(8, 2)","Bot_Move_(8, 5)","Bot_Move_(9, 1)","Bot_Move_(9, 5)","Bot_Move_(9, 6)"
Distance_from_bot_to_crew,1.000000,-0.471630,0.504405,-0.048295,-0.069090,-0.069090,-0.069090,-0.048295,-0.085619,-0.069090,...,-0.048295,0.718750,-0.048295,-0.048295,-0.048295,0.079545,-0.048295,-0.048295,-0.069090,-0.048295
Distance_from_bot_to_teleport,-0.471630,1.000000,0.335677,0.197606,0.150178,0.017668,0.150178,0.197606,0.186107,0.017668,...,-0.080278,-0.358161,-0.172905,-0.080278,-0.172905,0.104978,-0.080278,0.104978,0.017668,-0.080278
Distance_from_crew_to_teleport,0.504405,0.335677,1.000000,0.069154,0.219249,0.098929,-0.021390,0.069154,-0.026508,0.098929,...,-0.014952,0.237367,-0.099059,-0.014952,-0.267272,0.069154,-0.183165,0.153261,-0.141710,-0.014952
"Bot_Cell_(0, 3)",-0.048295,0.197606,0.069154,1.000000,-0.032513,-0.032513,-0.032513,-0.022727,-0.040291,-0.032513,...,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.032513,-0.022727
"Bot_Cell_(0, 4)",-0.069090,0.150178,0.219249,-0.032513,1.000000,-0.046512,-0.046512,-0.032513,-0.057639,-0.046512,...,-0.032513,-0.032513,-0.032513,-0.032513,-0.032513,-0.032513,-0.032513,-0.032513,-0.046512,-0.032513
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Bot_Move_(8, 2)",0.079545,0.104978,0.069154,-0.022727,-0.032513,-0.032513,-0.032513,-0.022727,-0.040291,-0.032513,...,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,1.000000,-0.022727,-0.022727,-0.032513,-0.022727
"Bot_Move_(8, 5)",-0.048295,-0.080278,-0.183165,-0.022727,-0.032513,-0.032513,-0.032513,-0.022727,-0.040291,-0.032513,...,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,1.000000,-0.022727,-0.032513,-0.022727
"Bot_Move_(9, 1)",-0.048295,0.104978,0.153261,-0.022727,-0.032513,-0.032513,-0.032513,-0.022727,-0.040291,-0.032513,...,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,1.000000,-0.032513,-0.022727
"Bot_Move_(9, 5)",-0.069090,0.017668,-0.141710,-0.032513,-0.046512,-0.046512,-0.046512,-0.032513,-0.057639,-0.046512,...,-0.032513,-0.032513,-0.032513,-0.032513,-0.032513,-0.032513,-0.032513,-0.032513,1.000000,-0.032513


In [147]:
correlation_matrix2

,Bot_Cell,Crew_Cell,Distance_from_bot_to_crew,Distance_from_bot_to_teleport,Distance_from_crew_to_teleport,Bot_Move
Bot_Cell,1.000000,0.567557,0.314177,-0.470888,-0.159452,0.847695
Crew_Cell,0.567557,1.000000,-0.325839,-0.217329,-0.602762,0.723933
Distance_from_bot_to_crew,0.314177,-0.325839,1.000000,-0.471630,0.504405,0.207873
Distance_from_bot_to_teleport,-0.470888,-0.217329,-0.471630,1.000000,0.335677,-0.484482
Distance_from_crew_to_teleport,-0.159452,-0.602762,0.504405,0.335677,1.000000,-0.283081
Bot_Move,0.847695,0.723933,0.207873,-0.484482,-0.283081,1.000000


TRAIN TEST SPLIT

In [139]:
encoded_data1 = one_hot_encoded_df.copy()
encoded_data2 = label_encoded_df.copy()
raw_data = df.copy()

In [140]:
X = encoded_data2.drop('Bot_Move', axis=1)
y = encoded_data2['Bot_Move']

In [141]:
X.head()

,Bot_Cell,Crew_Cell,Distance_from_bot_to_crew,Distance_from_bot_to_teleport,Distance_from_crew_to_teleport
0,27,2,8,1,7
1,25,3,7,1,8
2,23,2,4,3,7
3,20,8,3,5,6
4,17,7,3,6,5


In [100]:
print(type(y[0]))

<class 'numpy.int32'>


In [142]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

MODELS

In [148]:
model = LogisticRegression()
model.fit(X_train, y_train)


y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.0


C:\Users\shara\anaconda3\envs\MachineLearning\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
